In [2]:
from IPython.display import HTML
HTML("<style>code {background-color:#EEE !important; border:#CCC !important; padding:3px !important}</style>")

# Exploring the Nonlinearity Zoo

## TLDR

  1. This is a really small scale test. 1 ~75 Epoch run of cifar10, for each tested nonlinearity. Don't take any hard conclusions from this.
  
  2. The `ELU` shape is a smoothed rectifier. In the paper it asymptotes to `max(x,-1)`. This shift could be seen as an initializer for the biases, it's just much simpler to specify this way.
  
  4. Half of the training speed up seems to come from the improved initialization, and half from the smooth shape. 
  
  5. [A slightly positive initial bias to avoid "dead neurons"](https://www.tensorflow.org/versions/master/tutorials/mnist/pros/index.html#weight-initialization) is probably counter productive, at least initially (ReLU.shift in [this plot](#Compare-to-Glorot-2011)).

# Introduction

I find [Clevert et al. 2015: Fast and Accurate Deep Network Learning by Exponential Linear Units (ELUs)](http://arxiv.org/abs/1511.07289) surprising. So here I'm just trying a simple emperical comparison. 

Also it seems like there has been a zoo of alternate nonlinearities developing, and I'd like to see the effects myself.

It looks like Xu has already tried [RReLU's vs ELU with cifar100](https://gist.github.com/antinucleon/5de43ed2ff11b944f1bd). There is also his comparison of  ReLU, LReLU, RReLU, PReLU nonlinearities in [Xu et al. 2015](http://arxiv.org/abs/1505.00853). 

I'm doing this for two Reasons

### First
They claim a significant improvment for a, seemingly, small change.

### Second
In their introduction they reference [Glorot et al. 2011](http://jmlr.csail.mit.edu/proceedings/papers/v15/glorot11a/glorot11a.pdf), which investigates networks of: rectified linear units (ReLU), rectified tanh units, and softplus units. 

Glorot's paper found, counterintuitively, that the softplus networks performed uniformly worst (the intuition being that the smooth gradient would be helpful).

The proposed ELU is not significantly different from the softplus in _shape_, so any improvement may just be from the reduced "bias-shift". Maybe `max(x,-1)` is all you need. Sometimes [fast & simple](http://deeplearning.net/software/theano/library/tensor/nnet/nnet.html#tensor.nnet.sigmoid) is enough.

Maybe the smooth gradient helps but the more positive activations of the softplus contributes to the bias-shift enough to kill the advantage.

## Nonlinearities
I tested a list of standard and modified nonlinearities. These were inserted into each Conv or FC layer of a [simple convolutional network](#Network-Architecture).

Dotted names like `ReLU.down` are shifted versions of standard nonlinearities. These are included help disentangle the effect of shape vs. position. 

In the following equations `x` is the linear activation calculated by each unit.

Name                           | Function
-----                          | --------
linear                         | `x`
tanh                           | `tanh(x)`
ReLU(Rectified Linear Unit)    | `max(x,0)`
ReLU.down                      | `ReLU(x+1)-1`, `max(x,-1)`
ReLU.shift                     | `ReLU(x+softplus2(0))`
LReLU (Leaky ReLU)             | `max(x,x/10)`
ELU (Exponential Linear Unit)  | `exp(x)-1 if x < 0 else x`
ELU.up                         | `ELU(x-1)+1`
Softplus2                      | `ln(1+exp(2*x))/2`, `softplus(2*x)/2`
Softplus2.down                 | `softplus2(x+1)-1`
PReLU (Parametric ReLU)        | `max(x,a*x)`
bilinear                       | `max(x,a*x+b)`
maxout                         | `max(x1,x2)`


Softplus is rescaled, as `softplus2`, so that the scale is similar to the ELU.

The last three nonlinearities contain extra trainable parameters. 

Notably absent is `sigmoid`, because I have been unable to get it to learn anything on this network. This is unsurprising (vanishing gradients) and is entierly described in ([Glorot 2011](http://jmlr.org/proceedings/papers/v9/glorot10a/glorot10a.pdf), and [Ioffe 2015](http://arxiv.org/abs/1502.03167v3)). 



### Untrainable Nonlinearities

The untrainable nonlinearities are shown in the plot below (sorry for the clutter):

<img src="nonlinearities.png">

### Trainable Nonlinearities

These are just progressively more general versions of the LReLU.

[`PReLU`](http://arxiv.org/pdf/1502.01852.pdf), is an LReLU with trainable leakage. 

`Bilinear`, here is a `PReLU` with trainable leakage and bias.

[`Maxout`](http://jmlr.csail.mit.edu/proceedings/papers/v28/goodfellow13.pdf) uses `N` times as many output channels, and then takes the max over those `N`, here I'm using `N=2` (~twice the calculations). The the maxout paper uses `N=5`. 

The two-channel `maxout` being used here can't represent anything that isn't already covered by `bilinear`, I just wanted to see if there was a difference in the training process.

## CIFAR-10

I've modified [tensorflow's cifar10 tutorial](http://www.tensorflow.org/tutorials/deep_cnn/index.html) so that we can easily swap nonlinearities. I wrote this before all the higher level TF libraries were available.

It's packaged so that the `cifar10` package is runnable (`__main__.py`).

This wraps `cifar10.train` and can be used to run a bunch of training rounds, with different nonlinearities, or run time.

    $> python -m cifar10 -h
    
    $> python -m cifar10 --max_train_hours 0.5 ReLU ReLU.down ELU
    
    $> python -m cifar10 --max_train_hours 2 

The `--max_train_hours` option is **per run**
If there are no nonlinearities listed (as trailing args) it defaults to **all of them**.

## Network Architecture

The network being trained in these plots is nothing fancy. It contains stacks of 3x3 convs with occasional 2x2/2 max pooling. It's not carefully optimized in any way. I took the tutorial network, made it deeper, and added the global average pooling just to [decouple the model from the image size](https://www.facebook.com/yann.lecun/posts/10152820758292143).

TODO: upgrade to a standard architecture, like NIN in [Xu et al. 2015](http://arxiv.org/abs/1505.00853), so the results are comparable. There are way to many 3x3 Convs for this task.

I used the `tf.AdamOptimizer`, with default parameters.

No weight decay, learning-rate annealing, data augmentation. All weights and biases  are initialized with `tf.truncated_normal(...,stddev=0.1)` (TODO: try with biases = 0). This is close to [He's Rectified Xavier Initialization](http://arxiv.org/pdf/1502.01852.pdf), and a little above standard linear-xavier, for the 3x3 conv layers. So if anything the initialization is biased towards the standard rectifiers, not the downshifted ones (TODO: recheck this). If this weren't close, the network would not be trainable at all.

All the conv layers have 32 output channels. All dropouts use `keep=0.5`, the FC layer shapes are 32x384 and 348x192.

The layout is in [`cifar10.model.Model`](cifar10/model.py):

<img src='cifar10/model.png'>

### TensorFlow construction notes
TODO: switch code to just use tf-slim

The only way I could get it to display nicely was to use a lot of `tf.name_scope`s 

I run the training and validation sets in parallel, off the same model parameters. This forces a clear separation of the code that initializes the `tf.Variables` and the code that executes the model. 

I like the pattern of objects holding variables (often created in `__init__`), and (pure-function) methods executing the calculations (or just `__call__` in this case). Objects hold state, functions don't. 

So the main "model" block below, contains all the variables. Successive calls to the model object create the "Train" and "Validate" inference stacks. 

At a higher level the model looks like this:

<img src='cifar10/layout.png'>

Note the separate input and inference paths, and how both inference blocks use the same model parameters.

## Early training runs

Each nonlinearity under consideration was inserted into the above network, and trained for 16 hours (cpu only, 4 cores). The testing/validation set was run in parallel. 

    $> python -m cifar10 --max_train_hours 16
    
maxout was run for longer to get a similar number of epochs    

## Plot Notes
- all figures are generated by `python plots.py`

- Graph scales are identical across all figures.

- Line styles (color and marker type) are consistant across graphs, for each nonlinearitiy. 

- Downward triangle markers are for nonlinearities with a horizontal asymptote at y=-1, upward triangles for y=0

- Solid lines are for the training set Cross Entropy, dashed for the test/cross-validation set Cross Entropy.

- I don't know why the training curves are so linear, on these `semilogx` plots.

- The reported errors are smoothed with an exponential moving average, `momentum = 0.998` per batch so the center of mass is 500 batches behind the shown location.

- If zero mean inputs are helpful to initial training, as suggested by the ELU paper, symetric functions like `tanh` and `linear` should train quickly at first. 


## Compare to [Glorot 2011](http://jmlr.csail.mit.edu/proceedings/papers/v15/glorot11a/glorot11a.pdf)
The first thing to check is that our results are comparable to Glorot's Paper, minus the 'rectified tanh'

So here is a plot of the training evolution of `ReLU` vs. `softplus2`. I also included `ELU.up` to emphasize that they're basically the same.

`ReLU` wins, no surprise.

But the [ELU paper](http://arxiv.org/abs/1511.07289) explains that average neuron values are important (average across samples, not across neurons). `softplus2` is strictly greater that `ReLU`. This suggests that `ReLU` performance can be degraded, closer to `softplus2` , by shifting a ReLU left so that `ReLU.shift(0) == softplus2(0)`. 

<img src='Glorot.png'>

For the initial part of training `linear` is the fastest. Only the ReLU's validation entropy is below linear's at the end of training. 

Shifting the ReLU, `ReLU.shift`, seems to degrade training. So maybe the higher average value of softplus relative to ReLU totally explains the performance difference, and not the shape, as was suggested by Glorot.

Shifting the ReLU nonlinearity left is equivalent to adding an initial bias. So [a slightly positive bias to avoid "dead neurons"](https://www.tensorflow.org/versions/master/tutorials/mnist/pros/index.html#weight-initialization) is likely counter productive, (at least initially). 

It's also interesting the way this `ReLU.shift` has no efect on the space of models that can be represented, but it degrades training.

This first plot is also a nice reminder of how powerful dropout is. It's nice to see how the **validation** error is lower than the **training** error for most of training.

## Compare to [Xu 2015](http://arxiv.org/abs/1505.00853)

LReLU, and PReLU (initilized to `a=0.1` like LReLU) are close to eachother and clearly ahead of the standard ReLU. LReLU and PReLU reach a cross-entropy of 0.3 digits at 30-40 epochs, compared to 70 for the ReUL.

<img src='Xu.png'>

## Compare to [Clevert 2015](http://arxiv.org/abs/1511.07289)
Initially the `ELU,(a=1)` beats everything shown so far. `PReLU` catches up at 30 epochs.  

But shifting `softplus2` down to a horizontal asymptote of -1 gives indistinguishable performance. Both are clearly ahead of the standard `ReLU`, reaching a CE of 0.3 around 35 epochs, while `ReLU.down` is in betwen the two (0.3 digits at 50 epochs). `Tanh` learns quickly at first (symmetry, zero bias-shift, like linear) but falls behind linear as the training progresses. 

<img src='Clevert.png'>

# Others

Adding more flexibility to the LReLU/PReLU family doesn't help much.

Bilinear (initialized to `max(x,-1)`) appears to keep up with `ELU`, both hit a cross-entropy of 0.3 digits just after 30 epochs, with `maxout` lagging initially.

<img src='Piecewise.png'>

# Further questions

The smoothness of the ELU seems to help, as does the shifted asymptote. The flexibility of the `PReLU` seems to give comparable (better?) performance. It could be interesting to try a piecewise-smooth `PReLU` maybe improving the performance further.

[Batch normalization](http://arxiv.org/abs/1502.03167) likely make the entire "bias-shift" issue moot as it re-centers and normalizes every batch. This encourages good learning not just from the initialization point but throughout training.